In [ ]:
#reproducability
from numpy.random import seed
seed(1+347823)
import tensorflow as tf
tf.random.set_seed(1+63493)

import numpy as np
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
import os
import pandas as pd
import datetime
from scipy import stats
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler


In [ ]:

def load_RM_GW_and_HYRAS_Data(ID):
    pathGW = "./01_GWdata/"
    pathHYRAS = "./00_HYRAS/"

    GWData = pd.read_csv(pathGW+ID+'_GW-Data.csv', 
                          parse_dates=['Date'],index_col=0, dayfirst = True, 
                          decimal = '.', sep=',')
    HYRASData = pd.read_csv(pathHYRAS+ID+'_Hyras_weekly.csv',
                            parse_dates=['Date'],index_col=0, dayfirst = True,
                            decimal = '.', sep=',')
    data = pd.merge(GWData, HYRASData, how='inner', left_index = True, right_index = True)

    return data


def split_data(data, GLOBAL_SETTINGS):
    '''
    split data in four parts: training, early stopping, optimization, and testing
    strict dates for testing and optimization, relative portions for training and early stopping of the rest
    according to the input sequence length the parts do overlap ("_ext"), so the model can make use of the full data
    '''
    dataset = data[(data.index < GLOBAL_SETTINGS["test_start"])]
    dataset2 = data[(data.index < GLOBAL_SETTINGS["opt_start"])]
    
    TrainingData = dataset2[0:round(0.9 * len(dataset2))]
    StopData = dataset2[round(0.9 * len(dataset2))+1:]
    StopData_ext = dataset2[round(0.9 * len(dataset2))+1-GLOBAL_SETTINGS["seq_length"]:] 
    OptData = data[(data.index >= GLOBAL_SETTINGS["opt_start"]) & (data.index < GLOBAL_SETTINGS["test_start"])] 
    OptData_ext = pd.concat([dataset2.iloc[-GLOBAL_SETTINGS["seq_length"]:], OptData], axis=0)                                             

    TestData = data[(data.index >= GLOBAL_SETTINGS["test_start"]) & (data.index <= GLOBAL_SETTINGS["test_end"])] 
    TestData_ext = pd.concat([dataset.iloc[-GLOBAL_SETTINGS["seq_length"]:], TestData], axis=0)                                             

    return TrainingData, StopData, StopData_ext, OptData, OptData_ext, TestData, TestData_ext


def to_supervised(data, GLOBAL_SETTINGS):
    '''establish sequence to value data format, function based on code from machinelearningmastery.com'''
    X, Y = list(), list()
    # step over the entire history one time step at a time
    for i in range(len(data)):
        # find the end of this pattern
        end_idx = i + GLOBAL_SETTINGS["seq_length"]
        # check if we are beyond the dataset
        if end_idx >= len(data):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = data[i:end_idx, 1:], data[end_idx, 0]
        X.append(seq_x)
        Y.append(seq_y)
    return np.array(X), np.array(Y)



def gwmodel(ini,GLOBAL_SETTINGS,X_train, Y_train,X_stop, Y_stop):
    '''build and train tensorflow model'''
    seed(ini+42)
    tf.random.set_seed(ini+42)
    inp = tf.keras.Input(shape=(GLOBAL_SETTINGS["seq_length"], X_train.shape[2]))
    cnn = tf.keras.layers.Conv1D(filters=GLOBAL_SETTINGS["filters"],
                                         kernel_size=GLOBAL_SETTINGS["kernel_size"],
                                         activation='relu',
                                         padding='same')(inp)
    cnn = tf.keras.layers.MaxPool1D(padding='same')(cnn)
    cnn = tf.keras.layers.Dropout(0.1)(cnn)
    cnn = tf.keras.layers.Flatten()(cnn)
    cnn = tf.keras.layers.Dense(GLOBAL_SETTINGS["dense_size"], activation='relu')(cnn)
    output1 = tf.keras.layers.Dense(1, activation='linear')(cnn)
    
    # tie together
    model = tf.keras.Model(inputs=inp, outputs=output1)
        
    optimizer = tf.keras.optimizers.Adam(learning_rate=GLOBAL_SETTINGS["learning_rate"], epsilon=10E-3, clipnorm=GLOBAL_SETTINGS["clip_norm"])
    
    model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])
    
    
    # early stopping
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=0, 
                                          patience=GLOBAL_SETTINGS['patience'],
                                          restore_best_weights = True)
    
    # fit network
    history = model.fit(X_train, Y_train, validation_data=(X_stop, Y_stop), epochs=GLOBAL_SETTINGS["epochs"],
                        verbose=0,batch_size=GLOBAL_SETTINGS["batch_size"], callbacks=[es])
    
    return model, history




In [ ]:

def bayesOpt_function(densesize, batchsize, filters):
    '''function that is called by the bayesian optimization module and which is repeated over and over again to determine 
     the performance depending on the hyperparameters '''
    
    print(ID+" (pp:{}) BayesOpt-Iteration {}".format(loc,len(optimizer.res)+1))
    
    #some HPs allow only int values
    densesize = 2**int(densesize)
    batchsize = 2**int(batchsize)
    filters = 2**int(filters)
    
    GLOBAL_SETTINGS['batch_size'] = batchsize
    GLOBAL_SETTINGS['dense_size'] = densesize
    GLOBAL_SETTINGS['filters'] = filters
    print('  densesize {}, batchsize {}, filters {}'.format(densesize,batchsize,filters))

    ## load data
    data = load_RM_GW_and_HYRAS_Data(ID)
    
    #scale data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler_gwl = MinMaxScaler(feature_range=(0, 1))
    scaler_gwl.fit(pd.DataFrame(data['GWL']))
    data_n = pd.DataFrame(scaler.fit_transform(data), index=data.index, columns=data.columns)

    #split data
    TrainingData, StopData, StopData_ext, OptData, OptData_ext, TestData, TestData_ext = split_data(data, GLOBAL_SETTINGS)
    TrainingData_n, StopData_n, StopData_ext_n, OptData_n, OptData_ext_n, TestData_n, TestData_ext_n = split_data(data_n, GLOBAL_SETTINGS)
    
    #sequence data
    X_train, Y_train = to_supervised(TrainingData_n.values, GLOBAL_SETTINGS)
    X_stop, Y_stop = to_supervised(StopData_ext_n.values, GLOBAL_SETTINGS) 
    X_opt, Y_opt = to_supervised(OptData_ext_n.values, GLOBAL_SETTINGS)
    X_test, Y_test = to_supervised(TestData_ext_n.values, GLOBAL_SETTINGS) 

    #build and train model with inimax different initializations
    
    inimax = 5
    optresults_members = np.zeros((len(X_opt), inimax))
    for ini in range(inimax):
        print('    Member: '+str(ini))
        model,history = gwmodel(ini,GLOBAL_SETTINGS,X_train, Y_train, X_stop, Y_stop)  
        opt_sim_n = model.predict(X_opt)
        opt_sim = scaler_gwl.inverse_transform(opt_sim_n)
        optresults_members[:, ini] = opt_sim.reshape(-1,)
        
        fig = pyplot.figure(figsize=(5,1))
        pyplot.plot(history.history['loss'])
        pyplot.plot(history.history['val_loss'])
        pyplot.title(ID)
        pyplot.show()
        
    # calculate performance
    opt_sim_mean = np.mean(optresults_members,axis = 1)    
    sim = np.asarray(opt_sim_mean.reshape(-1,1))
    obs = np.asarray(scaler_gwl.inverse_transform(Y_opt.reshape(-1,1)))
    err = sim-obs
    meanTrainingGWL = np.mean(np.asarray(TrainingData['GWL']))
    meanStopGWL = np.mean(np.asarray(StopData['GWL']))
    err_nash = obs - np.mean([meanTrainingGWL, meanStopGWL])
    
    try:
        r = stats.pearsonr(sim[:,0], obs[:,0])
        r = r[0] #r
    except:
        r = [np.nan, np.nan]
        r = r[0] #r
    NSE = (1 - ((np.sum(err ** 2)) / (np.sum((err_nash) ** 2))))
    # score = NSE + r
   
    #plot optimization result
    pyplot.figure(figsize=(7,2))
    pyplot.plot(obs,color='k',linewidth=1)
    for ini in range(inimax):
        pyplot.plot(optresults_members[:,ini],color='r',linewidth=0.5,alpha = 0.5)
    pyplot.plot(sim,color='r',linewidth=1)
    pyplot.title(ID+', NSE '+str(np.round(NSE,3))+', r '+str(np.round(r,3)))
    pyplot.show()
    
    score = np.mean(err**2)
    print("score = "+str(np.round(score,4)))
    return -score



In [ ]:

def simulate_testset():
    '''mostly similar to the bayesOpt_function() function,
      but used to take the best model and calculate errors on 
      the test set instead of the optimization set'''
    
    ## load data
    data = load_RM_GW_and_HYRAS_Data(ID)
    
    #scale data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler_gwl = MinMaxScaler(feature_range=(0, 1))
    scaler_gwl.fit(pd.DataFrame(data['GWL']))
    data_n = pd.DataFrame(scaler.fit_transform(data), index=data.index, columns=data.columns)

    #split data
    TrainingData, StopData, StopData_ext, OptData, OptData_ext, TestData, TestData_ext = split_data(data, GLOBAL_SETTINGS)
    TrainingData_n, StopData_n, StopData_ext_n, OptData_n, OptData_ext_n, TestData_n, TestData_ext_n = split_data(data_n, GLOBAL_SETTINGS)
    
    #sequence data
    X_train, Y_train = to_supervised(TrainingData_n.values, GLOBAL_SETTINGS)
    X_stop, Y_stop = to_supervised(StopData_ext_n.values, GLOBAL_SETTINGS) 
    X_opt, Y_opt = to_supervised(OptData_ext_n.values, GLOBAL_SETTINGS)
    X_test, Y_test = to_supervised(TestData_ext_n.values, GLOBAL_SETTINGS) 

    #build and train model with inimax different initializations
    inimax = 20
    sim_members = np.zeros((len(X_test), inimax))
    sim_members[:] = np.nan
    
    model_path = './Results_XAI/models/'+ID+"/" # to save the model
    
    f = open('./Results_XAI/traininghistory_CNN_'+ID+'.txt', "w") # to save additional information on the training permanently
    if os.path.isdir(model_path) == False:
            os.mkdir(model_path)

    fig, axs = pyplot.subplots(inimax,1, figsize = (7,24))
    for ini in range(inimax):
        model,history = gwmodel(ini,GLOBAL_SETTINGS,X_train, Y_train, X_stop, Y_stop)  
        model.save(model_path+str(ini))
        loss = np.zeros((1, GLOBAL_SETTINGS['epochs']))
        loss[:,:] = np.nan
        loss[0,0:np.shape(history.history['loss'])[0]] = history.history['loss']
        val_loss = np.zeros((1, GLOBAL_SETTINGS['epochs']))
        val_loss[:,:] = np.nan
        val_loss[0,0:np.shape(history.history['val_loss'])[0]] = history.history['val_loss']
        print('loss', file = f)
        print(loss.tolist(), file = f)
        print('val_loss', file = f)
        print(val_loss.tolist(), file = f)
        
        axs[ini].plot(history.history['loss'],label = 'training')
        axs[ini].plot(history.history['val_loss'],label = 'validation')
        axs[ini].title.set_text('ini '+str(ini))

        test_sim = model.predict(X_test)
        test_sim = scaler_gwl.inverse_transform(test_sim)
        sim_members[:,ini]=test_sim.reshape(-1,)
    f.close()
    pyplot.tight_layout()
    pyplot.savefig('./Results_XAI/loss_'+ID+'.png')
    pyplot.show()
    
    # use this part to apply already existing models (need to comment the training and saving part above)
    # for ini in range(inimax):    
    #     model = tf.keras.models.load_model(model_path+str(ini))
    #     test_sim = model.predict(X_test)
    #     test_sim = scaler_gwl.inverse_transform(test_sim)
    #     sim_members[:,ini]= test_sim.reshape(-1,)
    
    #plot results
    fig, axs = pyplot.subplots(10,2, figsize = (14,24))
    col = 0
    for ini in range(inimax):
        row = ini
        if ini > 9:
            col = 1
            row = ini-10
        sim = sim_members[:,ini]
        axs[row,col].plot(TestData.index, sim, 'r', label ="simulated", linewidth = 1.7)
        axs[row,col].plot(TestData.index, np.asarray(scaler_gwl.inverse_transform(Y_test.reshape(-1,1))),
                    'k', label ="observed", linewidth=1.7,alpha=0.9)
        axs[row,col].title.set_text("Testfit, ini"+str(ini))
        axs[row,col].grid(visible=True, which='major', color='#666666', alpha = 0.3, linestyle='-')
    pyplot.tight_layout()
    pyplot.savefig('./Results_XAI/Members_test_'+ID+'.png')
    pyplot.show()        
        
    sim_mean = np.nanmean(sim_members,axis = 1)
    sim_uncertainty = [np.quantile(sim_members, 0.05, axis=1),np.quantile(sim_members, 0.95, axis=1)]
    
    #introduce GWL t-1 as additional Input (nalso needed for persistency index calculation)
    GWData_shift1 = pd.DataFrame(data['GWL'])
    GWData_shift1.index = GWData_shift1.index.shift(periods = 7, freq = 'D')
    GWData_shift1.rename(columns={"GWL": "GWLt-1"},inplace=True)
    PIData = GWData_shift1[(GWData_shift1.index >= GLOBAL_SETTINGS["test_start"]) & (GWData_shift1.index <= GLOBAL_SETTINGS["test_end"])]
    
    # get scores
    sim = np.asarray(sim_mean.reshape(-1,1))
    obs = np.asarray(scaler_gwl.inverse_transform(Y_test.reshape(-1,1)))
    obs_PI = np.asarray(PIData).reshape(-1,1)
    err = sim-obs
    err_rel = (sim-obs)/(np.max(data['GWL'])-np.min(data['GWL']))
    err_nash = obs - np.mean(np.asarray(data['GWL'][(data.index < GLOBAL_SETTINGS["test_start"])]))
    err_PI = obs-obs_PI
    
    NSE = 1 - ((np.sum(err ** 2)) / (np.sum((err_nash) ** 2)))
    try:
        r = stats.pearsonr(sim[:,0], obs[:,0])
        r = r[0] #r
    except:
        r = [np.nan, np.nan]
        r = r[0] #r
    R2 = r ** 2
    RMSE =  np.sqrt(np.mean(err ** 2))
    rRMSE = np.sqrt(np.mean(err_rel ** 2)) * 100
    Bias = np.mean(err)
    rBias = np.mean(err_rel) * 100
    PI = 1 - ((np.sum(err ** 2)) / (np.sum((err_PI) ** 2)))
    alpha = np.std(sim)/np.std(obs)
    beta = np.mean(sim)/np.mean(obs)
    KGE = 1-np.sqrt((r-1)**2+(alpha-1)**2+(beta-1)**2) #KGE
    
    scores = pd.DataFrame(np.array([[NSE, KGE, R2, RMSE, rRMSE, Bias, rBias, PI]]),
                   columns=['NSE', 'KGE','R2','RMSE','rRMSE','Bias','rBias','PI'])
    print(scores)
    
    return scores, TestData, sim, obs, sim_uncertainty


In [ ]:

class newJSONLogger(JSONLogger) :
      '''enables that the logger continues an already existing file, wich
        is also possible in a different way using the lates releases of 
        the bayesian optimization library (see documentation).
      '''
      
      def __init__(self, path):
            self._path=None
            super(JSONLogger, self).__init__()
            self._path = path if path[-5:] == ".json" else path + ".json"
            


In [ ]:

#%%
well_list = pd.read_csv("./locations.csv",sep=';',header = 0,encoding = 'Latin1')
    
for loc in [0]:#range(well_list.shape[0]): # loop all locations
    
    seed(1)
    tf.random.set_seed(1)

    
    ID = well_list.alias[loc]
    data = load_RM_GW_and_HYRAS_Data(ID)
    
    #define hyperparameters and other important stuff    
    GLOBAL_SETTINGS = {
        'clip_norm': True,
        'epochs': 3, #max number of training epochs to allow
        'patience': 30, # early stopping patience
        'learning_rate': 1e-3,
        'seq_length': 52, #length of the data input sequence for the model (here: 52 weeks = 1 year)
        'kernel_size': 3,
        'opt_start': pd.to_datetime('01012015', format='%d%m%Y'), 
        'test_start': pd.to_datetime('01012017', format='%d%m%Y'),
        'test_end': pd.to_datetime('31122020', format='%d%m%Y')
    }
    
    # if not all locations have data until the "test_end" data, you may need something like this
    if GLOBAL_SETTINGS["test_end"] > data.index[-1]:
        GLOBAL_SETTINGS["test_end"] = data.index[-1]
        GLOBAL_SETTINGS["test_start"] = GLOBAL_SETTINGS["test_end"] - datetime.timedelta(days=(365*4))
        GLOBAL_SETTINGS["opt_start"] = GLOBAL_SETTINGS["test_start"] - datetime.timedelta(days=(365*2))
    
    # define bounds for hyperparameters for optimization, here the numbers x are translated into 2^x later
    pbounds = {'densesize': (4,8),
                'batchsize': (4,9),
                'filters': (4,9)}
    
    optimizer = BayesianOptimization(
        f= bayesOpt_function, #function to optimize
        pbounds=pbounds, #parameter bounds
        random_state=1, 
        verbose = 0 # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent, verbose = 2 prints everything
        )
    
    # you may want to conitue by loading an existing optimizer
    log_already_available = 0
    if os.path.isfile("./Results_XAI/logs_CNN_"+ID+".json"):
        load_logs(optimizer, logs=["./Results_XAI/logs_CNN_"+ID+".json"]);
        print("\nExisting optimizer is already aware of {} points. (pp={})".format(len(optimizer.space),loc))
        log_already_available = 1
        
    # Save progress
    logger = newJSONLogger(path="./Results_XAI/logs_CNN_"+ID+".json")
    optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
    
    if log_already_available == 0:

        optimizer.maximize(init_points=0, n_iter=0)
        print(optimizer.max)# uses probe
        optimizer.maximize(
                init_points=20, #steps of random exploration (random starting points before bayesopt(?))
                n_iter=0, # steps of bayesian optimization
                acq="ei",# ei  = expected improvmenet (probably the most common acquisition function) 
                xi=0.05  #  Prefer exploitation (xi=0.0) / Prefer exploration (xi=0.1)
                )
    
    #define counters that control the number of optimization steps
    counter1 = 50#80 # min number of steps
    counter2 = 1#25 # between min and max, stop after counter2 steps if no improvement
    counter3 = 55#200 # max number of steps

    #check for progress and which step is the currently best step during optimization
    current_step = len(optimizer.res)
    beststep = False
    step = -1
    while not beststep:
        step = step + 1
        beststep = optimizer.res[step] == optimizer.max

    
    while current_step < counter1: 
            current_step = len(optimizer.res)
            beststep = False
            step = -1
            while not beststep:
                step = step + 1
                beststep = optimizer.res[step] == optimizer.max
            print("\nbeststep {}, current step {}".format(step+1, current_step+1))
            optimizer.maximize(
                init_points=0, #steps of random exploration / you can also start with counter1 random steps instead of the while loop
                n_iter=1, # steps of bayesian optimization
                acq="ei",# ei  = expected improvmenet (probably the most common acquisition function) 
                xi=0.05  #  Prefer exploitation (xi=0.0) / Prefer exploration (xi=0.1)
                )

    while (step + counter2 > current_step and current_step < counter3): # 
            current_step = len(optimizer.res)
            beststep = False
            step = -1
            while not beststep:
                step = step + 1
                beststep = optimizer.res[step] == optimizer.max
                
            print("\nbeststep {}, current step {}".format(step+1, current_step+1))
            optimizer.maximize(
                init_points=0, #steps of random exploration 
                n_iter=1, # steps of bayesian optimization
                acq="ei",# ei  = expected improvmenet (probably the most common acquisition function) 
                xi=0.05  #  Prefer exploitation (xi=0.0) / Prefer exploration (xi=0.1)
                )
        
    print("\nBEST:\t{}".format(optimizer.max))

    
    #get best values from optimizer
    densesize = optimizer.max.get("params").get("densesize")
    batchsize = optimizer.max.get("params").get("batchsize")
    filters = optimizer.max.get("params").get("filters")
        
    densesize = 2**int(densesize)
    batchsize = 2**int(batchsize)
    filters = 2**int(filters)
    
    GLOBAL_SETTINGS['batch_size'] = batchsize
    GLOBAL_SETTINGS['dense_size'] = densesize
    GLOBAL_SETTINGS['filters'] = filters
    
    #run test set simulations
    scores, TestData, sim, obs, sim_uncertainty = simulate_testset()
    
    #plot result
    pyplot.figure(figsize=(20,6))
    
    lb = sim_uncertainty[0]
    ub = sim_uncertainty[1]
    
    pyplot.fill_between(TestData.index, lb,
                ub, facecolor = (1,0.7,0,0.5),
                label ='90% confidence',linewidth = 1,
                edgecolor = (1,0.7,0,0.7))
    
    pyplot.plot(TestData.index, sim, 'r', label ="simulated mean", linewidth = 1.7)
    
    pyplot.plot(TestData.index, obs, 'k', label ="observed", linewidth=1.7,alpha=0.9)
    
    pyplot.title("CNN Model Test: "+ID, size=17,fontweight = 'bold')
    pyplot.ylabel('GWL [m asl]', size=15)
    pyplot.xlabel('Date',size=15)
    pyplot.legend(fontsize=15,bbox_to_anchor=(1.18, 1),loc='upper right',fancybox = False, framealpha = 1, edgecolor = 'k')
    pyplot.tight_layout()
    pyplot.grid(visible=True, which='major', color='#666666', alpha = 0.3, linestyle='-')
    pyplot.xticks(fontsize=14)
    pyplot.yticks(fontsize=14)
    
    s = """NSE = {:.2f}\nKGE = {:.2f}\nR²  = {:.2f}\nRMSE = {:.2f}\nrRMSE = {:.2f}
Bias = {:.2f}\nrBias = {:.2f}\n\nfilters = {:d}\ndense-size = {:d}
batchsize = {:d}\n""".format(scores.NSE[0],scores.KGE[0],scores.R2[0],
    scores.RMSE[0],scores.rRMSE[0],scores.Bias[0],scores.rBias[0],
    filters, densesize,batchsize)
    
    pyplot.figtext(0.872, 0.18, s, bbox=dict(facecolor='white'),fontsize = 15)
    pyplot.savefig('./Results_XAI/Test_'+ID+'_CNN.png', dpi=300,bbox_inches='tight')            
    pyplot.show()
    
    # print scores and results
    scores.to_csv('./Results_XAI/scores_'+ID+'.txt',float_format='%.3f',index=False)
    printdf = pd.DataFrame(data=np.c_[obs,sim,lb,ub],index=TestData.index)
    printdf = printdf.rename(columns={0: 'Obs', 1: 'Sim', 2:'lb:0.05', 3:'ub:95'})
    printdf.to_csv('./Results_XAI/results_'+ID+'.txt',sep=';', float_format = '%.6f')